# Problem: RST-DT files aren't tokenized, their corresponding PTB files are

* off-the-shelf tokenizers don't reproduce PTB tokenization exactly
* let's try the preprocessing provided by the [Educational Testing Service](https://github.com/EducationalTestingService)'s (ETS) [RST discourse parser](https://github.com/EducationalTestingService/discourse-parsing)

## ETS discourse-parser

* offers a script `convert_rst_discourse_tb`, in `discourseparsing.convert_rst_discourse_tb:main`,  
  which fixes the preprocesses the RST-DT files, so that nltk's tokenizer produces  
  the same tokenization that PTB uses
* NOTE: their code uses Python 3

Their code includes:

* some magic preprocessing

```python
with open(path_dis) as f:
    rst_tree_str = f.read().strip()
    rst_tree_str = fix_rst_treebank_tree_str(rst_tree_str)
    rst_tree_str = convert_parens_in_rst_tree_str(rst_tree_str)
    rst_tree = ParentedTree.fromstring(rst_tree_str)
    reformat_rst_tree(rst_tree)
```

* some file mappings

```python
# file mapping from the RSTDTB documentation
file_mapping = {'file1.edus': 'wsj_0764.out.edus',
                'file2.edus': 'wsj_0430.out.edus',
                'file3.edus': 'wsj_0766.out.edus',
                'file4.edus': 'wsj_0778.out.edus',
                'file5.edus': 'wsj_2172.out.edus'}
```

* some whitespace trickery

```python
edu = re.sub(r'>\s*', r'', edu).replace('&amp;', '&')
edu = re.sub(r'---', r'--', edu)
edu = edu.replace('. . .', '...')
...
```

* and a lot of "annoying edge cases"

```python
if path_basename == 'file1.edus':
    edu = edu.replace('founded by',
                      'founded by his grandfather.')
elif (path_basename == 'wsj_0660.out.edus'
      or path_basename == 'wsj_1368.out.edus'
      or path_basename == "wsj_1371.out.edus"):
    edu = edu.replace('S.p. A.', 'S.p.A.')
elif path_basename == 'wsj_1329.out.edus':
    edu = edu.replace('G.m.b. H.', 'G.m.b.H.')
...
```

# TODO: parse all files with discoursegraphs, then fix the "annoying edge cases"

In [1]:
import os
import re

from nltk.tree import ParentedTree
import sexpdata

import discoursegraphs as dg

RSTDT_ROOTDIR = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0')
RSTDT_TEST_FILE = os.path.join(RSTDT_ROOTDIR, 'TEST', 'wsj_1306.out.dis')

In [2]:
_ptb_paren_mapping = {'(': r'-LRB-',
                      ')': r'-RRB-',
                      '[': r'-LSB-',
                      ']': r'-RSB-',
                      '{': r'-LCB-',
                      '}': r'-RCB-'}

In [3]:
def fix_rst_treebank_tree_str(rst_tree_str):
    '''
    This removes some unexplained comments in two files that cannot be parsed.
    - data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2353.out.dis
    - data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2367.out.dis
    '''
    return re.sub(r'\)//TT_ERR', ')', rst_tree_str)

In [4]:
def convert_parens_in_rst_tree_str(rst_tree_str):
    '''
    This converts any brackets and parentheses in the EDUs of the RST discourse
    treebank to look like Penn Treebank tokens (e.g., -LRB-),
    so that the NLTK tree API doesn't crash when trying to read in the
    RST trees.
    '''
    for bracket_type, bracket_replacement in _ptb_paren_mapping.items():
        rst_tree_str = \
            re.sub('(_![^_(?=!)]*)\\{}([^_(?=!)]*_!)'.format(bracket_type),
                   '\\g<1>{}\\g<2>'.format(bracket_replacement),
                   rst_tree_str)
    return rst_tree_str

In [5]:
def disfile2tree(dis_filepath):
    with open(dis_filepath) as f:
        rst_tree_str = f.read().strip()
        rst_tree_str = fix_rst_treebank_tree_str(rst_tree_str)
        rst_tree_str = convert_parens_in_rst_tree_str(rst_tree_str)
        return ParentedTree.fromstring(rst_tree_str)    

In [7]:
tree = disfile2tree(RSTDT_TEST_FILE)

In [24]:
SUBTREE_TYPES = ('Root', 'Nucleus', 'Satellite')
NODE_TYPES = ('leaf', 'span')

def is_tree(tree):
    """returns true, iff the given element is a (sub)tree (and not a leaf or span node)"""
    return tree.label() in SUBTREE_TYPES

In [37]:
def get_node_type(tree):
    """returns the node type (leaf or span) of a subtree (i.e. Nucleus or Satellite)"""
    node_type = tree[0].label()
    assert node_type in ('leaf', 'span')
    return node_type

In [57]:
class Mock(object):
    ns = 'rst'

def get_node_id(self, nuc_or_sat):
    node_type = get_node_type(nuc_or_sat)
    if node_type == 'leaf':
        leaf_id = nuc_or_sat[0].leaves()[0]
        return '{}:{}'.format(self.ns, leaf_id)
    else: # node_type == 'span'
        span_start = nuc_or_sat[0].leaves()[0]
        span_end = nuc_or_sat[0].leaves()[1]
        return '{}:span:{}-{}'.format(self.ns, span_start, span_end)

In [60]:
mock = Mock()

print is_tree(tree), get_node_type(tree), get_node_id(mocky, subelem)
for subelem in tree:
    if is_tree(subelem):
        print is_tree(subelem), get_node_type(subelem), get_node_id(mock, subelem)

True span rst:span:21-47
True span rst:span:1-20
True span rst:span:21-47


'rst'

In [68]:
def treeprint(tree, tab=0):
    for elem in tree:
        if is_tree(elem):
            print "{}{}".format('   '*tab, get_node_id(mock, elem))
            treeprint(elem, tab=tab+1)

In [69]:
treeprint(tree)

rst:span:1-20
   rst:span:1-14
      rst:span:1-8
         rst:span:1-4
            rst:span:1-3
               rst:1
               rst:span:2-3
                  rst:2
                  rst:3
            rst:4
         rst:span:5-8
            rst:span:5-6
               rst:5
               rst:6
            rst:span:7-8
               rst:7
               rst:8
      rst:span:9-14
         rst:span:9-11
            rst:9
            rst:span:10-11
               rst:10
               rst:11
         rst:span:12-14
            rst:12
            rst:span:13-14
               rst:13
               rst:14
   rst:span:15-20
      rst:span:15-17
         rst:15
         rst:span:16-17
            rst:16
            rst:17
      rst:span:18-20
         rst:span:18-19
            rst:18
            rst:19
         rst:20
rst:span:21-47
   rst:span:21-30
      rst:span:21-27
         rst:span:21-23
            rst:21
            rst:span:22-23
               rst:22
               rst:23
  

In [49]:
span = tree[1][0]

In [51]:
span.leaves()

['1', '20']

In [15]:
# for folder in ('TEST', 'TRAINING'):
#     for rstfile in dg.util.find_files(os.path.join(RSTDT_ROOTDIR, folder), '*.dis'):
#         with open(rstfile) as f:
#             dg.read_dis(rstfile)